# Spark SQL: Advanced operations

## Introduction

In this example, we load and query multiple CSV files, converted from the XML data dumps of the Italian community in StackExchange. We illustrate some advanced operations of the structured data API, such as performing JOIN operations efficiently between different tables.

Documentation about data format can be found here: https://ia800107.us.archive.org/27/items/stackexchange/readme.txt.

Code for this example has been borrowed in part from: https://github.com/spark-in-action/first-edition/blob/master/ch05/python/ch05-listings.py.

## Preparation

In this case, the files are already converted from the original XML dump files provided by SE. Thus, there is no need to load third-party libraries to read these data.

In [2]:
# Load external packages programatically
import os
# packages = "com.databricks:spark-xml_2.11:0.5.0"

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

# os.environ["PYSPARK_SUBMIT_ARGS"] = (
#     "--packages {0} pyspark-shell".format(packages)
# )

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("StackExchange Italian Job")
    .getOrCreate() )
sc = spark.sparkContext

23/03/25 13:53:25 WARN Utils: Your hostname, helium resolves to a loopback address: 127.0.1.1; using 10.6.36.17 instead (on interface wlo1)
23/03/25 13:53:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/25 13:53:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# print(spark)
# print(sc)
spark

In [ ]:
!head -n 20 ../data/SE/italianPosts.csv

In [7]:
from pyspark.sql import Row
from datetime import datetime

def toIntSafe(inval):
    try:
        return int(inval)
    except ValueError:
        return None

def toTimeSafe(inval):
    try:
        return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S.%f")
    except ValueError:
        return None

# def toLongSafe(inval):
#     try:
#         return long(inval)
#     except ValueError:
#         return None

def stringToPost(row):
    r = row.split("~")
    return Row(
        toIntSafe(r[0]),
        toTimeSafe(r[1]),
        toIntSafe(r[2]),
        r[3],
        toIntSafe(r[4]),
        toTimeSafe(r[5]),
        toIntSafe(r[6]),
        toIntSafe(r[7]),
        r[8],
        toIntSafe(r[9]),
        toIntSafe(r[10]),
        toIntSafe(r[11]),
        int(r[12])
    )

In [8]:
from pyspark.sql.types import *
from pyspark.sql import functions as func

# Define data scheme for posts
postSchema = StructType([
  StructField("commentCount", IntegerType(), True),
  StructField("lastActivityDate", TimestampType(), True),
  StructField("ownerUserId", LongType(), True),
  StructField("body", StringType(), True),
  StructField("score", IntegerType(), True),
  StructField("creationDate", TimestampType(), True),
  StructField("viewCount", IntegerType(), True),
  StructField("title", StringType(), True),
  StructField("tags", StringType(), True),
  StructField("answerCount", IntegerType(), True),
  StructField("acceptedAnswerId", LongType(), True),
  StructField("postTypeId", LongType(), True),
  StructField("id", LongType(), False)
])

In [10]:
# Preprocess data and load to DataFrame using schema above
itPostsRows = sc.textFile("../data/SE/italianPosts.csv")
rowRDD = itPostsRows.map(lambda x: stringToPost(x))

itPostsDFStruct = spark.createDataFrame(rowRDD, postSchema)

In [11]:
# Total number of users in this excerpt from complete SciFi dump
itPostsDFStruct.count()

1261

In [12]:
# Get list of column names
itPostsDFStruct.columns

['commentCount',
 'lastActivityDate',
 'ownerUserId',
 'body',
 'score',
 'creationDate',
 'viewCount',
 'title',
 'tags',
 'answerCount',
 'acceptedAnswerId',
 'postTypeId',
 'id']

In [13]:
# Get list of tuples (colname, type)
itPostsDFStruct.dtypes

[('commentCount', 'int'),
 ('lastActivityDate', 'timestamp'),
 ('ownerUserId', 'bigint'),
 ('body', 'string'),
 ('score', 'int'),
 ('creationDate', 'timestamp'),
 ('viewCount', 'int'),
 ('title', 'string'),
 ('tags', 'string'),
 ('answerCount', 'int'),
 ('acceptedAnswerId', 'bigint'),
 ('postTypeId', 'bigint'),
 ('id', 'bigint')]

In [14]:
# Print DataFrame complete content schema
itPostsDFStruct.printSchema()

root
 |-- commentCount: integer (nullable = true)
 |-- lastActivityDate: timestamp (nullable = true)
 |-- ownerUserId: long (nullable = true)
 |-- body: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- creationDate: timestamp (nullable = true)
 |-- viewCount: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- answerCount: integer (nullable = true)
 |-- acceptedAnswerId: long (nullable = true)
 |-- postTypeId: long (nullable = true)
 |-- id: long (nullable = false)



In [15]:
itPostsDFStruct.take(5)

[Row(commentCount=4, lastActivityDate=datetime.datetime(2013, 11, 11, 18, 21, 10, 903000), ownerUserId=17, body="&lt;p&gt;The infinitive tense is commonly used for expressing rules especially in signs (of any kind, not just road signs).&lt;/p&gt;&lt;p&gt;For instance&lt;/p&gt;&lt;blockquote&gt;  &lt;p&gt;Non fumare&lt;br&gt;  Non calpestare il prato&lt;br&gt;  Tenere la destra&lt;/p&gt;&lt;/blockquote&gt;&lt;p&gt;The language &quot;trick&quot; behind this use of the infinitive form is the omission of the clause &lt;em&gt;Si prega di&lt;/em&gt; or equivalent, so the above sentences are read as&lt;/p&gt;&lt;blockquote&gt;  &lt;p&gt;&lt;em&gt;&lt;strong&gt;Si prega di&lt;/em&gt;&lt;/strong&gt; non fumare&lt;br&gt;  &lt;strong&gt;&lt;em&gt;Si prega di&lt;/em&gt;&lt;/strong&gt; non calpestare il prato&lt;br&gt;  &lt;strong&gt;&lt;em&gt;Si prega di&lt;/em&gt;&lt;/strong&gt; tenere la destra&lt;/p&gt;&lt;/blockquote&gt;&lt;p&gt;Such form is not used in everyday's spoken language, as it's a co

## Initial queries

In [17]:
# Explore some contents in several interesting columns
# Include values connecting with other tables
(itPostsDFStruct.select("id", "postTypeId", "ownerUserId", "creationDate",
                       "tags", "viewCount", "answerCount", "score")
                .filter("postTypeId = 1")
                .show(10))

+----+----------+-----------+--------------------+--------------------+---------+-----------+-----+
|  id|postTypeId|ownerUserId|        creationDate|                tags|viewCount|answerCount|score|
+----+----------+-----------+--------------------+--------------------+---------+-----------+-----+
|1166|         1|         12|2013-11-10 19:44:...| &lt;word-choice&gt;|       61|          1|    1|
|1168|         1|        154|2013-11-10 22:03:...|&lt;english-compa...|      187|          4|   11|
|1173|         1|         63|2013-11-11 10:28:...|&lt;usage&gt;&lt;...|      122|          3|    5|
|1175|         1|         63|2013-11-11 11:31:...|&lt;usage&gt;&lt;...|      114|          2|    4|
|1176|         1|         63|2013-11-11 11:39:...|&lt;usage&gt;&lt;...|       58|          2|    3|
|1179|         1|         63|2013-11-11 12:58:...|&lt;usage&gt;&lt;...|       60|          2|    3|
|1183|         1|         22|2013-11-11 14:43:...|&lt;history&gt;&l...|      321|          1|    6|


In [18]:
# Filter content: count num. of questions including "Italiano" in their body
# Note the use of func.instr inside the filter method
itPostsDFStruct.filter(func.instr(itPostsDFStruct["body"], "Italiano") > 0).count()

46

### Exercise 1: List the 10 most recently modified questions

In [20]:
# Write your solution here
(itPostsDFStruct.select("id", "postTypeId", "ownerUserId", "creationDate",
                        "lastActivityDate", "tags", "viewCount",
                        "answerCount", "score")
 # .orderBy("lastActivityDate", ascending=False)
 .orderBy(itPostsDFStruct.lastActivityDate.desc())
 .show(10)
)

+----+----------+-----------+--------------------+--------------------+--------------------+---------+-----------+-----+
|  id|postTypeId|ownerUserId|        creationDate|    lastActivityDate|                tags|viewCount|answerCount|score|
+----+----------+-----------+--------------------+--------------------+--------------------+---------+-----------+-----+
|2343|         1|        707|2014-09-12 09:44:...|2014-09-13 22:25:...|&lt;orthography&g...|      102|          1|    3|
|2330|         1|        193|2014-09-09 12:34:...|2014-09-13 13:40:...|&lt;vocabulary&gt...|       88|          1|    4|
|2337|         2|        193|2014-09-11 01:16:...|2014-09-13 13:40:...|                    |     null|       null|    1|
|2345|         1|        707|2014-09-13 08:55:...|2014-09-13 13:29:...|   &lt;etymology&gt;|       47|          1|    3|
|2347|         2|         37|2014-09-13 13:10:...|2014-09-13 13:29:...|                    |     null|       null|    4|
|2346|         1|        193|201

**Ejercicio**: Recuperar posts con `lastActivityDate` más reciente que el 1 de diciembre de 2013 y con score > 5.

In [21]:
(itPostsDFStruct.select("id", "postTypeId", "ownerUserId",
                        "lastActivityDate", "tags", "viewCount",
                        "answerCount", "score")
.filter((itPostsDFStruct.lastActivityDate > '2013-12-01') & (itPostsDFStruct.score > 5))
.orderBy(itPostsDFStruct.lastActivityDate)
).show()

+----+----------+-----------+--------------------+--------------------+---------+-----------+-----+
|  id|postTypeId|ownerUserId|    lastActivityDate|                tags|viewCount|answerCount|score|
+----+----------+-----------+--------------------+--------------------+---------+-----------+-----+
|1432|         2|          8|2013-12-01 09:13:...|                    |     null|       null|    6|
|1395|         2|        193|2013-12-01 23:12:...|                    |     null|       null|    7|
|1439|         2|          5|2013-12-02 08:27:...|                    |     null|       null|    8|
|1442|         2|         70|2013-12-04 20:59:...|                    |     null|       null|    6|
|1444|         2|         87|2013-12-06 22:16:...|                    |     null|       null|    6|
|1449|         2|        193|2013-12-07 14:12:...|                    |     null|       null|    8|
|1462|         2|         70|2013-12-09 00:03:...|                    |     null|       null|    6|


## SQL Functions for calculations

There are 4 types of SQL functions we can use with structured data in Spark:

* **Scalar functions**: Return a single value for each row, making calculations on one or more columns.
* **Aggregate functions**: Return a single value for a group of rows.
* **Window functions**: Return several values for a group of rows.
* **User-defined functions**: Include custom scalar or aggregate functions, created by the user.

### Scalar and aggregate functions

They can operate on numerical values (e.g. math operations), string values (string operations) or date/time/datetime values (date-time operations).

**Example: Retrieve the body of the question that was active for the longest time interval?**

In [20]:
oldest_question = (itPostsDFStruct.filter(itPostsDFStruct.postTypeId == 1)
 .withColumn("activePeriod",
             func.datediff(itPostsDFStruct.lastActivityDate,
                           itPostsDFStruct.creationDate))
 .orderBy(func.desc("activePeriod"))
)
print("Body of the oldest question:", oldest_question.head().body.replace("&lt;","<").replace("&gt;",">"))

Body of the oldest question: <p>The plural of <em>braccio</em> is <em>braccia</em>, and the plural of <em>avambraccio</em> is <em>avambracci</em>.</p><p>Why are the plural of those words so different, if they both are referring to parts of the human body, and <em>avambraccio</em> derives from <em>braccio</em>?</p>


Another example: Calculate the average and maximum score of all questions, and the total number of questions. In this case, we give an alias to all result columns, and we also round the average score result.

In [23]:
itPostsDFStruct.select(func.round(func.avg(itPostsDFStruct.score),4).alias("average_score"),
                       func.max(itPostsDFStruct.score).alias("max_score"),
                       func.min(itPostsDFStruct.score).alias("min_score"),
                       func.count(itPostsDFStruct.score).alias("total_count")).show()

+-------------+---------+---------+-----------+
|average_score|max_score|min_score|total_count|
+-------------+---------+---------+-----------+
|       4.1594|       24|       -3|       1261|
+-------------+---------+---------+-----------+



### Window functions

Window functions can also calculate summary results, but instead of producing a single row result per group they let us define a "moving group" of row (named *frames*), with respect to the current row. This moving group can be used to calculate different values to be combined with the values of the current row.

To indicate the column on which we want to define the window we use the function `over(window_def)`.

Some window functions currently available are:

* `first`: Returns the value in the first row in the frame.
* `last`: Returns the value in the first row in the frame.
* `lag`: Pivot operation, returns the value in the row that is `offset` rows behind the current row in the frame. Return `default` if the row does not exist.
* `lead`: Pivot operation, returns the value in the row that is `offset` rows before the current row in the frame. Return `default` if the row does not exist.
* `ntile`: returns the ntile group id (from 1 to n inclusive) in an ordered window partition. For example, if n is 4, the first quarter of the rows will get value 1, the second quarter will get 2, the third quarter will get 3, and the last quarter will get 4. This is equivalent to the `NTILE` function in SQL.
* `cume_dist`: Fraction of rows in the frame whose value is less than or equal to the value in the row being processed.
* `rank`: Returns the rank (by value) of the row in the frame.
* `dense_rank`: Returns the rank (by value) of the row in the frame, but it gives the same rank to rows with tied values.
* `percent_rank`: Returns rank of the row, divided by total number of rows in the frame.
* `row_number`: Returns the sequential number of the row in the frame.

**Example: Find the maximum score of all user questions (`postTypeId=1`) and, for each question, how much its score is below the maximum score for that user.**

*Strategy for the query*: Retrieve the maxium score of all user questions and, for each question, how much its score is below the maximum score for that user.

In [24]:
from pyspark.sql.window import Window

winDf = (itPostsDFStruct.filter(itPostsDFStruct.postTypeId == 1)
         .select(itPostsDFStruct.id,
                 itPostsDFStruct.ownerUserId,itPostsDFStruct.acceptedAnswerId,
                 itPostsDFStruct.score,
                 (func.max(itPostsDFStruct.score)
                  .over(Window.partitionBy(itPostsDFStruct.ownerUserId)).alias("maxPerUser"))
                )
        )
winDf.withColumn("toMax", winDf.maxPerUser - winDf.score).show(20)

+----+-----------+----------------+-----+----------+-----+
|  id|ownerUserId|acceptedAnswerId|score|maxPerUser|toMax|
+----+-----------+----------------+-----+----------+-----+
|1637|          4|            null|    7|         7|    0|
|1192|          8|            1212|   13|        20|    7|
|1276|          8|            null|    6|        20|   14|
|1321|          8|            1336|   10|        20|   10|
|1365|          8|            1377|    3|        20|   17|
|   1|          8|               2|   20|        20|    0|
| 112|          8|            null|    7|        20|   13|
|1166|         12|            null|    1|        14|   13|
|1187|         12|            null|    4|        14|   10|
|1189|         12|            null|    8|        14|    6|
|1197|         12|            null|    8|        14|    6|
|1211|         12|            null|    5|        14|    9|
|1217|         12|            null|    4|        14|   10|
|1239|         12|            null|    4|        14|   1

**Example: Calculate, for each question, the `id` of the next and previous question created by its owner, by creation date.**

*Strategy for the query*: Filter posts with `postTypeId==1`. Use the `lag` and `lead` functions to retrieve the previous and next rows in the frame (defined as all questions of that user). The window is the same for both operations (lag and lead): the partitioning is done by `ownerUserId` and you must order by `creationDate` the questions in the frame. Finally, order the results by `ownerUserId` and `questionId` to make it clearer.

In [25]:
(itPostsDFStruct.filter(itPostsDFStruct.postTypeId == 1)
 .select(itPostsDFStruct.id, itPostsDFStruct.ownerUserId, itPostsDFStruct.creationDate,
         (func.lag(itPostsDFStruct.id, 1)
          .over(Window.partitionBy(itPostsDFStruct.ownerUserId)
                .orderBy(itPostsDFStruct.creationDate)).alias("prev")
         ), 
         (func.lead(itPostsDFStruct.id, 1)
          .over(Window.partitionBy(itPostsDFStruct.ownerUserId)
                .orderBy(itPostsDFStruct.creationDate)).alias("next"))
        )
 .orderBy(itPostsDFStruct.ownerUserId, itPostsDFStruct.id).show()
)

+----+-----------+--------------------+----+----+
|  id|ownerUserId|        creationDate|prev|next|
+----+-----------+--------------------+----+----+
|1637|          4|2014-01-24 06:51:...|null|null|
|   1|          8|2013-11-05 20:22:...|null| 112|
| 112|          8|2013-11-08 13:14:...|   1|1192|
|1192|          8|2013-11-11 21:01:...| 112|1276|
|1276|          8|2013-11-15 16:09:...|1192|1321|
|1321|          8|2013-11-20 16:42:...|1276|1365|
|1365|          8|2013-11-23 09:09:...|1321|null|
|  11|         12|2013-11-05 21:30:...|null|  17|
|  17|         12|2013-11-05 22:17:...|  11|  18|
|  18|         12|2013-11-05 22:34:...|  17|  19|
|  19|         12|2013-11-05 22:38:...|  18|  63|
|  63|         12|2013-11-06 17:54:...|  19|  65|
|  65|         12|2013-11-06 18:07:...|  63|  69|
|  69|         12|2013-11-06 19:41:...|  65|  70|
|  70|         12|2013-11-06 20:35:...|  69|  89|
|  89|         12|2013-11-07 19:22:...|  70|  94|
|  94|         12|2013-11-07 20:42:...|  89| 107|


### User-defined functions (UDFs)

Sometimes, the specific function that we need to implement a calculation for our query is not provided in the standard programming API. In that case, we can define our own customized functions for that purpose. That kind of functions are called *UDFs* (User Defined Functions).

The method `functions.udf` let us create our own function, receiving a function that is previously defined with the required design and logic. A UDF must take zero or more columns as input (the maximum is 10 columns), and returns the final value.

For example, let's assume that we need a UDF to find how many tags are included in each question. Tags are written as concatenated strings surrounded by angle brackets (`&lt;` for `<` and `&gt;` for `>`).

In [26]:
# Create a new UDF named count_tags
count_tags = func.udf(lambda tags: tags.count("&lt;"), IntegerType())

# Use the new UDF in your query
(itPostsDFStruct.filter(itPostsDFStruct.postTypeId == 1)
 .select("tags", count_tags(itPostsDFStruct.tags).alias("tagCount")).show(10, False)
)

+-------------------------------------------------------------------+--------+
|tags                                                               |tagCount|
+-------------------------------------------------------------------+--------+
|&lt;word-choice&gt;                                                |1       |
|&lt;english-comparison&gt;&lt;translation&gt;&lt;phrase-request&gt;|3       |
|&lt;usage&gt;&lt;verbs&gt;                                         |2       |
|&lt;usage&gt;&lt;tenses&gt;&lt;english-comparison&gt;              |3       |
|&lt;usage&gt;&lt;punctuation&gt;                                   |2       |
|&lt;usage&gt;&lt;tenses&gt;                                        |2       |
|&lt;history&gt;&lt;english-comparison&gt;                          |2       |
|&lt;idioms&gt;&lt;etymology&gt;                                    |2       |
|&lt;idioms&gt;&lt;regional&gt;                                     |2       |
|&lt;grammar&gt;                                    

## Advanced queries

### Missing values

There are three possible options to deal with missing values:
* Drop all lines with missing values (`null` or `NaN`).
* Impute missing values in a column with a constant.
* Replace *certain* values with string or numeric constants.

Let's see some examples.

In [17]:
cleanPosts = itPostsDFStruct.na.drop()
cleanPosts.count()

0

In [18]:
itPostsDFStruct.na.fill({"viewCount": 0}).show(10)

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+
|           4|2013-11-11 18:21:...|         17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|        0| null|                    |       null|            null|         2|1165|
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61| null| &lt;word-choice&gt;|          1|            null|         1|1166|
|           2|2013-11-10 20:31:...|         17|&lt;p&gt;Il verbo...|    5|2013-11-10 19:58:...|        0| null

In [ ]:
itPostsDFStruct.na.replace(1177, 3000, ["id", "acceptedAnswerId"]).show(10)

### Grouping data and join operations

We can group data in a similar fashion as we do with the `GROUP BY` clause in SQL. It is enabled with the method `DataFrame.groupBy()`, that returns a `GroupedData` object (see: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData). `GroupedData` objects represent groups of rows that share the same values in the columns specified in the argument passed to `groupBy()`. As a result, we can apply severa operations (`avg`, `count`, `max`, `min`, `sum`) to perform aggregated calculations across the groups. The value returned by these functions is a `DataFrame` object with the columns affected by the `groupBy()` method, plus a new column with the results of the aggregated operation.

Example: Calculate the number of posts per author, associated tags (treating each combination of tags as a different value) and the post type.

In [14]:
# Create a new UDF named replace_angle_brackets
replace_angle_brackets = func.udf(lambda tags: tags.replace("&lt;", "<").replace("&gt;", ">"), 
                                  StringType())

# Perform the query including the new UDF
(itPostsDFStruct.groupBy(itPostsDFStruct.ownerUserId,
                         replace_angle_brackets(itPostsDFStruct.tags).alias("tags"),
                         itPostsDFStruct.postTypeId)
 .count()
 .orderBy(itPostsDFStruct.ownerUserId.desc())
 .show(10)
)

+-----------+--------------------+----------+-----+
|ownerUserId|                tags|postTypeId|count|
+-----------+--------------------+----------+-----+
|        862|                    |         2|    1|
|        855|         <resources>|         1|    1|
|        846|<translation><eng...|         1|    1|
|        845|<word-meaning><tr...|         1|    1|
|        842|  <verbs><resources>|         1|    1|
|        835|    <grammar><verbs>|         1|    1|
|        833|                    |         2|    1|
|        833|           <meaning>|         1|    1|
|        833|<meaning><article...|         1|    1|
|        814|                    |         2|    1|
+-----------+--------------------+----------+-----+
only showing top 10 rows



Example: Calculate the last activity date and the maximum score per user. In the second query, we map the column name to the function(s) that we want to apply on its values.

In [ ]:
# The following two queries are equivalent

(itPostsDFStruct
 .groupBy(itPostsDFStruct.ownerUserId)
 .agg(func.max(itPostsDFStruct.lastActivityDate).alias("max_last_activity_date"),
      func.max(itPostsDFStruct.score).alias("max_score"))
 .orderBy("max_score", ascending=False)
 .show(10)
)
# (itPostsDFStruct
#  .groupBy(itPostsDFStruct.ownerUserId)
#  .agg({"lastActivityDate": "max", "score": "max"})
#  .orderBy("ownerUserId")
#  .show(10)
# )

### The `rollup()` and `cube()` functions

Two additional functions that perform more complicated aggregate operations are `rollup()` and `cube()`. The allow us to calculate intermediate aggregation values for each value of the first column in the aggregation, so that they always respect the hierarchy of the input columns.

See https://github.com/spark-in-action/first-edition/blob/master/ch05/python/ch05-listings.py for example use cases for these functions.

### Join operations

Example: Combine the posts and votes tables from SE on the `postId` column.

In [28]:
# Read and load the table with votes info
itVotesRaw = sc.textFile("data/SE/italianVotes.csv").map(lambda x: x.split("~"))
itVotesRows = itVotesRaw.map(lambda row: Row(id=int(row[0]), postId=int(row[1]), voteTypeId=int(row[2]), creationDate=datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f")))
votesSchema = StructType([
  StructField("creationDate", TimestampType(), False),
  StructField("id", LongType(), False),
  StructField("postId", LongType(), False),
  StructField("voteTypeId", IntegerType(), False)
  ])  

votesDf = spark.createDataFrame(itVotesRows, votesSchema)

In [29]:
# Query 1 (inner join)
postsVotes = itPostsDFStruct.join(votesDf, itPostsDFStruct.id == votesDf.postId)
postsVotes.show(10)

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+----------------+-----------+----------------+----------+---+-------------------+----+------+----------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|title|            tags|answerCount|acceptedAnswerId|postTypeId| id|       creationDate|  id|postId|voteTypeId|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+----------------+-----------+----------------+----------+---+-------------------+----+------+----------+
|           1|2013-11-06 00:22:...|         18|&lt;p&gt;&quot;De...|    3|2013-11-06 00:05:...|     null| null|                |       null|            null|         2| 26|2013-11-06 00:00:00|  62|    26|         2|
|           1|2013-11-06 00:22:...|         18|&lt;p&gt;&quot;De...|    3|2013-11-06 00:05:...|     null| null|                |       n

In [30]:
# Query 2 (outer join)
postsVotesOuter = itPostsDFStruct.join(votesDf, itPostsDFStruct.id == votesDf.postId, "outer")
postsVotesOuter.show(10)

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+----------------+-----------+----------------+----------+---+-------------------+----+------+----------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|title|            tags|answerCount|acceptedAnswerId|postTypeId| id|       creationDate|  id|postId|voteTypeId|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+----------------+-----------+----------------+----------+---+-------------------+----+------+----------+
|           1|2013-11-06 00:22:...|         18|&lt;p&gt;&quot;De...|    3|2013-11-06 00:05:...|     null| null|                |       null|            null|         2| 26|2013-11-06 00:00:00|  62|    26|         2|
|           1|2013-11-06 00:22:...|         18|&lt;p&gt;&quot;De...|    3|2013-11-06 00:05:...|     null| null|                |       n

### Exercise: Perform 2 additional queries on the joint tables created before

In [ ]:
# Query 1

In [ ]:
# Query 2

## The End

In [8]:
# Remember to stop SparkContext before shutting down this notebook
sc.stop()